In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.options.display.max_info_columns=500
pd.options.display.max_info_rows=550
# from tensorflow.keras.losses import mean_absolute_percentage_error as mmape
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
# from sklearn.compose import ColumnTransformer
from sklearn import set_config; set_config(display='diagram')
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error

In [3]:
from carbon_forecast.utils.preproc import load_n_preproc_all_avail
df = load_n_preproc_all_avail()
X = df.drop(columns=["company_name", "TCUID", "year", "intensity_1and2","intensity_scope3"])
y = df["intensity_1and2"]

Reading data...
Preproc on power % revenue
Preproc on Sector Revenue
Selecting the year 2013 required...
Dropping rows that are showing NaN on...
-> intensity_scope1CY2013
-> intensity_scope2CY2013
-> intensity_scope3CY2013
-> Revenue2013
Filling employee numbers with mean value... (subject to change later)
Filling EV with mean value... (subject to change later)
Calculating Employees & EV per Revenue...
Filling null Climate Strategy Score with 0...
Cleaning P/E Ratio...
Adding Scope 1 and Scope 2 data together...
Doing some column renaming...
Filling in Disclosure...
Dataset preproc-ed for the year 2013!👍
Reading data...
Preproc on power % revenue
Preproc on Sector Revenue
Selecting the year 2014 required...
Dropping rows that are showing NaN on...
-> intensity_scope1CY2014
-> intensity_scope2CY2014
-> intensity_scope3CY2014
-> Revenue2014
Filling employee numbers with mean value... (subject to change later)
Filling EV with mean value... (subject to change later)
Calculating Employees 

In [4]:
X.columns.tolist()

['Sector',
 'Employees / Revenue',
 'EV / Revenue',
 'climate_strategy_score',
 'disclosure',
 'pe_rat',
 'revenue',
 'nuclear_percentage_revenue',
 'thermal_coal_percentage_revenue',
 'ultra_deep_sea_drilling_percentage_revenue',
 'shale_oil_gas_percentage_revenue',
 'coal_percentage_revenue',
 'arctic_drilling_percentage_revenue',
 'oil_sands_percentage_revenue',
 'Abrasive product manufacturing',
 'Accounting, tax preparation, bookkeeping, and payroll services',
 'Adhesive manufacturing',
 'Advertising and related services',
 'Air and gas compressor manufacturing',
 'Air conditioning, refrigeration, and warm air heating equipment manufacturing',
 'Air purification and ventilation equipment manufacturing',
 'Air transportation',
 'Aircraft engine and engine parts manufacturing',
 'Aircraft manufacturing',
 'Alkalies and chlorine manufacturing',
 'All other basic inorganic chemical manufacturing',
 'All other chemical product and preparation manufacturing',
 'All other converted paper

In [5]:
from carbon_forecast.utils.tx import our_tx, tt_split
transformer = our_tx(X)
X_train_transformed, X_test_transformed, y_train, y_test = tt_split(X,y,transformer)
X_train_transformed

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
from carbon_forecast.utils.pca import our_pca
X_proj, X_test_proj = our_pca(X_train_transformed, X_test_transformed, 12)
X_proj

array([[ 5.90597312e-01, -5.08552911e-01,  9.89725990e-01, ...,
        -2.35128376e-02,  5.83733980e-02, -2.85706906e-01],
       [ 4.16288030e-01, -2.56176005e-01, -1.81792621e-01, ...,
        -7.16385054e-03,  8.01023430e-02, -2.64503248e-01],
       [-3.82123709e-01, -5.57040131e-02, -3.41239335e-02, ...,
        -2.35010629e-02,  8.11906951e-04, -8.78317856e-03],
       ...,
       [ 7.43391282e-01,  7.62657117e-01,  4.44439357e-02, ...,
        -1.70396923e-03,  9.20275005e-02, -2.69413589e-01],
       [-3.96119675e-01, -7.17362855e-02, -4.49078801e-02, ...,
        -7.59188559e-02, -3.12257846e-02, -4.17346421e-02],
       [-3.41484490e-01, -3.80065550e-01,  1.06404108e+00, ...,
        -3.22229532e-02, -3.72303700e-02, -5.97069160e-04]])

In [7]:
model = XGBRegressor(n_estimators=1000, max_depth=10, objective="reg:squarederror", eval_metric="mape")
cross_val_score(model, X_proj, y_train, cv=5, scoring="neg_mean_absolute_percentage_error")

array([-4.03340713, -2.20808091, -5.94809135, -7.94449283, -2.4584482 ])

In [8]:
model.fit(X_proj, y_train)
model

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='mape', gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [9]:
y_pred = model.predict(X_test_proj)


In [10]:
mean_absolute_percentage_error(y_test, y_pred)

3.093351862802589

In [12]:
model.save_model("c_int.json")

In [13]:
from sklearn.decomposition import PCA
pca=PCA(n_components=12)
pca.fit(X_train_transformed)

PCA(n_components=12)

In [14]:
file = open("c_int_pca.pkl", "wb")
pickle.dump(pca, file)
file.close()

In [11]:
# model = XGBRegressor(n_estimators=1000, max_depth=10, eta=0.2, reg_alpha=0, reg_lambda=1.3, subsample=0.9, colsample_bytree =0.8, objective="reg:squarederror", eval_metric="mape")
# cross_val_score(model, X_proj, y_train, cv=5, scoring="neg_mean_absolute_percentage_error")

In [15]:
secrev_list = ['Abrasive product manufacturing',
 'Accounting, tax preparation, bookkeeping, and payroll services',
 'Adhesive manufacturing',
 'Advertising and related services',
 'Air and gas compressor manufacturing',
 'Air conditioning, refrigeration, and warm air heating equipment manufacturing',
 'Air purification and ventilation equipment manufacturing',
 'Air transportation',
 'Aircraft engine and engine parts manufacturing',
 'Aircraft manufacturing',
 'Alkalies and chlorine manufacturing',
 'All other basic inorganic chemical manufacturing',
 'All other chemical product and preparation manufacturing',
 'All other converted paper product manufacturing',
 'All other crop farming',
 'All other food manufacturing',
 'All other forging, stamping, and sintering',
 'All other miscellaneous electrical equipment and component manufacturing',
 'All other miscellaneous manufacturing',
 'All other miscellaneous professional, scientific, and technical services',
 'All other miscellaneous wood product manufacturing',
 'All other paper bag and coated and treated paper manufacturing',
 'All other petroleum and coal products manufacturing',
 'All other textile product mills',
 'All other transportation equipment manufacturing',
 'Alumina refining and primary aluminum production',
 'Aluminum product manufacturing from purchased aluminum',
 'Ammunition manufacturing',
 'Amusement parks, arcades, and gambling industries',
 'Analytical laboratory instrument manufacturing',
 'Animal (except poultry) slaughtering, rendering, and processing',
 'Animal production, except cattle and poultry and eggs',
 'Apparel accessories and other apparel manufacturing',
 'Apparel knitting mills',
 'Apparel, Piece Goods, and Notions Wholesalers',
 'Architectural, engineering, and related services',
 'Arms, ordnance, and accessories manufacturing',
 'Artificial and synthetic fibers and filaments manufacturing',
 'Asphalt paving mixture and block manufacturing',
 'Asphalt shingle and coating materials manufacturing',
 'Audio and video equipment manufacturing',
 'Automatic environmental control manufacturing',
 'Automobile manufacturing',
 'Automotive equipment rental and leasing',
 'Automotive repair and maintenance, except car washes',
 'Ball and roller bearing manufacturing',
 'Bare printed circuit board manufacturing',
 'Bauxite Mining',
 'Beet sugar manufacturing',
 'Biological product (except diagnostic) manufacturing',
 'Biomass Power Generation',
 'Bituminous Coal Underground Mining',
 'Bituminous Coal and Lignite Surface Mining',
 'Blind and shade manufacturing',
 'Boat building',
 'Book publishers',
 'Bowling centers',
 'Bread and bakery product manufacturing',
 'Breakfast cereal manufacturing',
 'Breweries',
 'Brick, tile, and other structural clay product manufacturing',
 'Broadcast and wireless communications equipment',
 'Broadwoven fabric mills',
 'Broom, brush, and mop manufacturing',
 'Building Material and Garden Equipment and Supplies Dealers',
 'Business support services',
 'Cable and other subscription programming',
 'Car washes',
 'Carbon and graphite product manufacturing',
 'Carbon black manufacturing',
 'Carpet and rug mills',
 'Cattle ranching and farming',
 'Cement manufacturing',
 'Cheese manufacturing',
 'Child day care services',
 'Chocolate and confectionery manufacturing from cacao beans',
 'Clay and nonclay refractory manufacturing',
 'Clothing and Clothing Accessories Stores',
 'Coal Power Generation',
 'Coated and laminated paper, packaging paper and plastics film manufacturing',
 'Coating, engraving, heat treating and allied activities',
 'Coffee and tea manufacturing',
 'Commercial and industrial machinery and equipment rental and leasing',
 'Commercial and industrial machinery and equipment repair and maintenance',
 'Communication and energy wire and cable manufacturing',
 'Community food, housing, and other relief services, including rehabilitation services',
 'Computer storage device manufacturing',
 'Computer systems design services',
 'Computer terminals and other computer peripheral equipment manufacturing',
 'Concrete pipe, brick, and block manufacturing',
 'Confectionery manufacturing from purchased chocolate',
 'Construction machinery manufacturing',
 'Cookie, cracker, and pasta manufacturing',
 'Copper Mining',
 'Copper rolling, drawing, extruding and alloying',
 'Cotton farming',
 'Couriers and messengers',
 'Crown and closure manufacturing and metal stamping',
 'Crude Petroleum and Natural Gas Extraction',
 'Curtain and linen mills',
 'Custom architectural woodwork and millwork manufacturing',
 'Custom computer programming services',
 'Cut and sew apparel contractors',
 'Cut stone and stone product manufacturing',
 'Cutlery, utensil, pot, and pan manufacturing',
 'Cutting tool and machine tool accessory manufacturing',
 'Dairy cattle and milk production',
 'Data processing, hosting, and related services',
 'Death care services',
 'Dental equipment and supplies manufacturing',
 'Directory, mailing list, and other publishers',
 'Distilleries',
 'Dog and cat food manufacturing',
 'Doll, toy, and game manufacturing',
 'Drilling oil and gas wells',
 'Dry, condensed, and evaporated dairy product manufacturing',
 'Dry-cleaning and laundry services',
 'Electric Bulk Power Transmission and Control',
 'Electric Power Distribution',
 'Electric lamp bulb and part manufacturing',
 'Electrical and Electronic Goods Wholesalers',
 'Electricity and signal testing instruments manufacturing',
 'Electromedical and electrotherapeutic apparatus manufacturing',
 'Electron tube manufacturing',
 'Electronic and precision equipment repair and maintenance',
 'Electronic capacitor, resistor, coil, transformer, and other inductor manufacturing',
 'Electronic computer manufacturing',
 'Electronic connector manufacturing',
 'Electronics and Appliance Stores',
 'Elementary and secondary schools',
 'Employment services',
 'Engineered wood member and truss manufacturing',
 'Environmental and other technical consulting services',
 'Fabric coating mills',
 'Fabricated pipe and pipe fitting manufacturing',
 'Facilities support services',
 'Farm machinery and equipment manufacturing',
 'Fats and oils refining and blending',
 'Ferrous metal foundries',
 'Fertilizer manufacturing',
 'Fiber, yarn, and thread mills',
 'Fishing',
 'Fitness and recreational sports centers',
 'Flat glass manufacturing',
 'Flavoring syrup and concentrate manufacturing',
 'Flour milling and malt manufacturing',
 'Fluid milk and butter manufacturing',
 'Fluid power process machinery',
 'Food services and drinking places',
 'Food, Beverage, Health, and Personal Care Stores',
 'Footwear manufacturing',
 'Forest nurseries, forest products, and timber tracts',
 'Frozen food manufacturing',
 'Fruit and vegetable canning, pickling, and drying',
 'Fruit farming',
 'Funds, trusts, and other financial vehicles',
 'Furniture and Home Furnishings Stores',
 'Gasket, packing, and sealing device manufacturing',
 'Gasoline Stations',
 'General Merchandise Stores',
 'General and consumer goods rental except video tapes and discs',
 'Geothermal Power Generation',
 'Glass container manufacturing',
 'Glass product manufacturing made of purchased glass',
 'Gold Ore Mining',
 'Grain farming',
 'Grantmaking, giving, and social advocacy organizations',
 'Greenhouse, nursery, and floriculture production',
 'Grocery and Related Product Wholesalers',
 'Ground or treated mineral and earth manufacturing',
 'Guided missile and space vehicle manufacturing',
 'Handtool manufacturing',
 'Hardware manufacturing',
 'Heating equipment (except warm air furnaces) manufacturing',
 'Heavy duty truck manufacturing',
 'Home health care services',
 'Hospitals',
 'Hotels and motels, including casino hotels',
 'Household cooking appliance manufacturing',
 'Household laundry equipment manufacturing',
 'Household refrigerator and home freezer manufacturing',
 'Hydroelectric Power Generation',
 'Ice cream and frozen dessert manufacturing',
 'In-vitro diagnostic substance manufacturing',
 'Independent artists, writers, and performers',
 'Individual and family services',
 'Industrial gas manufacturing',
 'Industrial mold manufacturing',
 'Industrial process furnace and oven manufacturing',
 'Industrial process variable instruments manufacturing',
 'Institutional furniture manufacturing',
 'Insurance agencies, brokerages, and related activities',
 'Insurance carriers',
 'Internet publishing and broadcasting',
 'Internet service providers and web search portals',
 'Investigation and security services',
 'Iron and steel mills and ferroalloy manufacturing',
 'Iron ore mining',
 'Irradiation apparatus manufacturing',
 'Jewelry and silverware manufacturing',
 'Junior colleges, colleges, universities, and professional schools',
 'Knit fabric mills',
 'Laminated plastics plate, sheet (except packaging), and shape manufacturing',
 'Landfill Gas Power Generation',
 'Lawn and garden equipment manufacturing',
 'Lead Ore and Zinc Ore Mining',
 'Leather and hide tanning and finishing',
 'Legal services',
 'Lessors of nonfinancial intangible assets',
 'Light truck and utility vehicle manufacturing',
 'Lighting fixture manufacturing',
 'Lime and gypsum product manufacturing',
 'Logging',
 'Lumber and Other Construction Materials Wholesalers',
 'Magnetic and optical recording media manufacturing',
 'Management of companies and enterprises',
 'Management, scientific, and technical consulting services',
 'Manufactured home (mobile home) manufacturing',
 'Material handling equipment manufacturing',
 'Mattress manufacturing',
 'Mechanical power transmission equipment manufacturing',
 'Medical and diagnostic labs and outpatient and other ambulatory care services',
 'Medicinal and botanical manufacturing',
 "Men's and boys' cut and sew apparel manufacturing",
 'Metal and other household furniture manufacturing',
 'Metal can, box, and other metal container (light gauge) manufacturing',
 'Metal cutting and forming machine tool manufacturing',
 'Metal tank (heavy gauge) manufacturing',
 'Military armored vehicle, tank, and tank component manufacturing',
 'Mineral wool manufacturing',
 'Mining and oil and gas field machinery manufacturing',
 'Miscellaneous Durable Goods Wholesalers',
 'Miscellaneous Nondurable Goods Wholesalers',
 'Miscellaneous Store Retailers',
 'Miscellaneous nonmetallic mineral products',
 'Monetary authorities and depository credit intermediation',
 'Motion picture and video industries',
 'Motor Vehicle and Machinery, Equipment, and Supplies Wholesalers',
 'Motor Vehicle and Parts Dealers',
 'Motor and generator manufacturing',
 'Motor home manufacturing',
 'Motor vehicle body manufacturing',
 'Motor vehicle parts manufacturing',
 'Motorcycle, bicycle, and parts manufacturing',
 'Museums, historical sites, zoos, and parks',
 'Musical instrument manufacturing',
 'Narrow fabric mills and schiffli machine embroidery',
 'Natural Gas Liquid Extraction',
 'Natural Gas Power Generation',
 'Natural gas distribution',
 'Newspaper publishers',
 'Nickel Mining',
 'Nonchocolate confectionery manufacturing',
 'Nondepository credit intermediation and related activities',
 'Nonferrous metal (except copper and aluminum) rolling, drawing, extruding and alloying',
 'Nonferrous metal foundries',
 'Nonresidential commercial and health care structures',
 'Nonresidential maintenance and repair',
 'Nonresidential manufacturing structures',
 'Nonstore Retailers',
 'Nonupholstered wood household furniture manufacturing',
 'Nonwoven fabric mills',
 'Nuclear Electric Power Generation',
 'Nursing and residential care facilities',
 'Office administrative services',
 'Office furniture manufacturing',
 'Office supplies (except paper) manufacturing',
 'Offices of physicians, dentists, and other health practitioners',
 'Oilseed farming',
 'Ophthalmic goods manufacturing',
 'Optical instrument and lens manufacturing',
 'Ornamental and architectural metal products manufacturing',
 'Other Electric Power Generation',
 'Other Metal Ore Mining',
 'Other accommodations',
 'Other aircraft parts and auxiliary equipment manufacturing',
 'Other amusement and recreation industries',
 'Other animal food manufacturing',
 'Other basic organic chemical manufacturing',
 'Other commercial and service industry machinery manufacturing',
 'Other communications equipment manufacturing',
 'Other computer related services, including facilities management',
 'Other concrete product manufacturing',
 'Other cut and sew apparel manufacturing',
 'Other educational services',
 'Other electronic component manufacturing',
 'Other engine equipment manufacturing',
 'Other fabricated metal manufacturing',
 'Other general purpose machinery manufacturing',
 'Other industrial machinery manufacturing',
 'Other information services',
 'Other leather and allied product manufacturing',
 'Other major household appliance manufacturing',
 'Other nonmetallic mineral mining and quarrying',
 'Other nonresidential structures',
 'Other personal services',
 'Other plastics product manufacturing',
 'Other pressed and blown glass and glassware manufacturing',
 'Other residential structures',
 'Other rubber product manufacturing',
 'Other support services',
 'Packaging machinery manufacturing',
 'Paint and coating manufacturing',
 'Paper mills',
 'Paperboard Mills',
 'Paperboard container manufacturing',
 'Performing arts companies',
 'Periodical publishers',
 'Personal and household goods repair and maintenance',
 'Personal care services',
 'Pesticide and other agricultural chemical manufacturing',
 'Petrochemical manufacturing',
 'Petroleum Power Generation',
 'Petroleum lubricating oil and grease manufacturing',
 'Petroleum refineries',
 'Petroleum, Chemical, and Allied Products Wholesalers',
 'Pharmaceutical preparation manufacturing',
 'Photographic and photocopying equipment manufacturing',
 'Photographic services',
 'Pipeline transportation',
 'Plastics and rubber industry machinery manufacturing',
 'Plastics bottle manufacturing',
 'Plastics material and resin manufacturing',
 'Plastics packaging materials and unlaminated film and sheet manufacturing',
 'Plastics pipe and pipe fitting manufacturing',
 'Plate work and fabricated structural product manufacturing',
 'Plumbing fixture fitting and trim manufacturing',
 'Polystyrene foam product manufacturing',
 'Postal service',
 'Pottery, ceramics, and plumbing fixture manufacturing',
 'Poultry and egg production',
 'Poultry processing',
 'Power boiler and heat exchanger manufacturing',
 'Power, distribution, and specialty transformer manufacturing',
 'Power-driven handtool manufacturing',
 'Prefabricated wood building manufacturing',
 'Primary battery manufacturing',
 'Primary smelting and refining of copper',
 'Primary smelting and refining of nonferrous metal (except copper and aluminum)',
 'Printed circuit assembly (electronic assembly) manufacturing',
 'Printing',
 'Printing ink manufacturing',
 'Promoters of performing arts and sports and agents for public figures',
 'Propulsion units and parts for space vehicles and guided missiles',
 'Pulp mills',
 'Pump and pumping equipment manufacturing',
 'Radio and television broadcasting',
 'Rail transportation (Diesel)',
 'Rail transportation (Electric)',
 'Railroad rolling stock manufacturing',
 'Ready-mix concrete manufacturing',
 'Real estate',
 'Reconstituted wood product manufacturing',
 'Relay and industrial control manufacturing',
 'Residential maintenance and repair',
 'Residential permanent site single- and multi-family structures',
 'Rolling mill and other metalworking machinery manufacturing',
 'Rubber and plastics hoses and belting manufacturing',
 'Sand, gravel, clay, and ceramic and refractory minerals mining and quarrying',
 'Sanitary paper product manufacturing',
 'Sawmills and wood preservation',
 'Scientific research and development services',
 'Seafood product preparation and packaging',
 'Search, detection, and navigation instruments manufacturing',
 'Seasoning and dressing manufacturing',
 'Secondary smelting and alloying of aluminum',
 'Securities, commodity contracts, investments, and related activities',
 'Semiconductor and related device manufacturing',
 'Semiconductor machinery manufacturing',
 'Services to buildings and dwellings',
 'Ship building and repairing',
 'Showcase, partition, shelving, and locker manufacturing',
 'Sign manufacturing',
 'Small electrical appliance manufacturing',
 'Snack food manufacturing',
 'Soap and cleaning compound manufacturing',
 'Soft drink and ice manufacturing',
 'Software publishers',
 'Software, audio, and video media reproducing',
 'Solar Power Generation',
 'Sound recording industries',
 'Soybean and other oilseed processing',
 'Special tool, die, jig, and fixture manufacturing',
 'Specialized design services',
 'Spectator sports',
 'Speed changer, industrial high-speed drive, and gear manufacturing',
 'Sporting and athletic goods manufacturing',
 'Spring and wire product manufacturing',
 'Stationery product manufacturing',
 'Steel product manufacturing from purchased steel',
 'Stone mining and quarrying',
 'Storage battery manufacturing',
 'Sugar cane mills and refining',
 'Sugarcane and sugar beet farming',
 'Support activities for agriculture and forestry',
 'Support activities for oil and gas operations',
 'Support activities for other mining',
 'Support activities for printing',
 'Support activities for transportation',
 'Surgical and medical instrument manufacturing',
 'Surgical appliance and supplies manufacturing',
 'Switchgear and switchboard apparatus manufacturing',
 'Synthetic dye and pigment manufacturing',
 'Synthetic rubber manufacturing',
 'Tar Sands Extraction',
 'Telecommunications',
 'Telephone apparatus manufacturing',
 'Textile and fabric finishing mills',
 'Textile bag and canvas mills',
 'Tire manufacturing',
 'Tobacco product manufacturing',
 'Toilet preparation manufacturing',
 'Totalizing fluid meters and counting devices manufacturing',
 'Transit and ground passenger transportation',
 'Travel arrangement and reservation services',
 'Travel trailer and camper manufacturing',
 'Tree nut farming',
 'Truck trailer manufacturing',
 'Truck transportation',
 'Turbine and turbine generator set units manufacturing',
 'Turned product and screw, nut, and bolt manufacturing',
 'Unlaminated plastics profile shape manufacturing',
 'Upholstered household furniture manufacturing',
 'Uranium-Radium-Vanadium Ore Mining',
 'Urethane and other foam product (except polystyrene) manufacturing',
 'Valve and fittings other than plumbing',
 'Vegetable and melon farming',
 'Vending, commercial, industrial, and office machinery manufacturing',
 'Veneer and plywood manufacturing',
 'Veterinary services',
 'Warehousing and storage',
 'Waste management and remediation services',
 'Watch, clock, and other measuring and controlling device manufacturing',
 'Water transportation',
 'Water, sewage and other systems',
 'Wave & Tidal Power Generation',
 'Wet corn milling',
 'Wind Power Generation',
 'Wineries',
 'Wiring device manufacturing',
 "Women's and girls' cut and sew apparel manufacturing",
 'Wood container and pallet manufacturing',
 'Wood kitchen cabinet and countertop manufacturing',
 'Wood windows and doors and millwork']

In [16]:
for i,j in enumerate(secrev_list):
    if j == "Coal Power Generation":
        print(i)

78


In [22]:
transformer.get_feature_names_out()

array(['num_tr__Abrasive product manufacturing',
       'num_tr__Accounting, tax preparation, bookkeeping, and payroll services',
       'num_tr__Adhesive manufacturing',
       'num_tr__Advertising and related services',
       'num_tr__Air and gas compressor manufacturing',
       'num_tr__Air conditioning, refrigeration, and warm air heating equipment manufacturing',
       'num_tr__Air purification and ventilation equipment manufacturing',
       'num_tr__Air transportation',
       'num_tr__Aircraft engine and engine parts manufacturing',
       'num_tr__Aircraft manufacturing',
       'num_tr__Alkalies and chlorine manufacturing',
       'num_tr__All other basic inorganic chemical manufacturing',
       'num_tr__All other chemical product and preparation manufacturing',
       'num_tr__All other converted paper product manufacturing',
       'num_tr__All other crop farming',
       'num_tr__All other food manufacturing',
       'num_tr__All other forging, stamping, and sintering'

In [21]:
X_train_transformed[0]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      